In [21]:
import pandas as pd 
import numpy as np

#Matrix operations
from itertools import product

# Factors / CPD's

### Synthetic data

In [22]:
alpha = 0.5
nodes = [7, 8, 1, 10, 6, 9, 3, 5, 11, 4, 2]
parents_top = [[], [], [7, 8], [7], [7,10], [10,6], [9,8], [9,8], [9,8], [11, 8, 9], [4, 8]]

# Switch the from topological order to index order
def node_to_index(nodes, parents):
    reordered_parents = [-1] * (max(nodes) + 1)
    for node, parent in zip(nodes, parents):
        reordered_parents[node] = parent
    return reordered_parents[1:max(nodes)+1]
parents = node_to_index(nodes, parents_top)
parents.insert(0, -1)

def synthetic_cpd(alpha, parent):
    # Create all possible combinations of parent values
    df = pd.DataFrame(list(product([0, 1], repeat=len(parent))), columns=parent)
    df['prob'] = 1 - alpha ** (1 + df.sum(axis=1)) # P(X=1|parents)
    return df

df_x7 = synthetic_cpd(alpha, parents[7])
df_x8 = synthetic_cpd(alpha, parents[8])
df_x1 = synthetic_cpd(alpha, parents[1])
df_x10 = synthetic_cpd(alpha, parents[10])
df_x6 = synthetic_cpd(alpha, parents[6])
df_x9 = synthetic_cpd(alpha, parents[9])
df_x3 = synthetic_cpd(alpha, parents[3])
df_x5 = synthetic_cpd(alpha, parents[5])
df_x11 = synthetic_cpd(alpha, parents[11])
df_x4 = synthetic_cpd(alpha, parents[4])
df_x2 = synthetic_cpd(alpha, parents[2])

factor_list_top = [df_x7, df_x8, df_x1, df_x10, df_x6, df_x9, df_x3, df_x5, df_x11, df_x4, df_x2]
factor_list = node_to_index(nodes, factor_list_top)

# Sum product

In [23]:
def factor_product(factor_1, factor_2):
    nodes_to_join = factor_1.columns.intersection(factor_2.columns)
    nodes_to_join = nodes_to_join.drop("prob")
    nodes = list(nodes_to_join)

    new_factor = factor_1.merge(factor_2, how='inner', on=nodes)
    new_factor["prob"] = new_factor["prob_x"]*new_factor["prob_y"]
    new_factor = new_factor.drop(columns=["prob_x","prob_y"])
    return new_factor

factor_product(factor_list_top[4], factor_list_top[5])

,7,10,6,prob
0,0,0,0,0.250000
1,0,0,1,0.375000
2,1,0,0,0.375000
3,1,0,1,0.562500
4,0,1,0,0.562500
5,0,1,1,0.656250
6,1,1,0,0.656250
7,1,1,1,0.765625


In [28]:
def sum_product_eliminate_var(factors : list, var : int):
    lst_scope = []   
    lst_not_scope = []
    for factor in factors:
        if var in factor.columns:
            lst_scope.append(factor)
        else:
            lst_not_scope.append(factor)
    for elm in lst_scope:
        print(elm.columns)

    # Multiply all factors in scope
    factor_ = lst_scope[0]
    # print(factor_)
    for i in range(1, len(lst_scope)):
        factor_ = factor_product(factor_, lst_scope[i])

    

    return factor_

sum_product_eliminate_var(factor_list, 10)

Index([7, 10, 'prob'], dtype='object')
Index([10, 6, 'prob'], dtype='object')


,7,10,6,prob
0,0,0,0,0.250000
1,0,0,1,0.375000
2,1,0,0,0.375000
3,1,0,1,0.562500
4,0,1,0,0.562500
5,0,1,1,0.656250
6,1,1,0,0.656250
7,1,1,1,0.765625
